# build database

In [ ]:
from __future__ import print_function

from itertools import (islice, izip)

import arrow
import github3
import requests

from sqlalchemy import or_
from github_settings import (ry_username, ry_password,
                             username, password,
                             token, 
                             GITENBERG_GITHUB_TOKEN,
                             GITENBERG_TRAVIS_ACCESS_TOKEN,
                             RDHYEE_GITHUB_TOKEN,
                             RDHYEE_TRAVIS_ACCESS_TOKEN, 
                             RDHYEE_TRAVIS_PROFILE_TOKEN)


from second_folio import (apply_to_repos, all_repos)

from gitenberg_utils import (GitenbergJob,
                             GitenbergTravisJob,
                             ForkBuildRepo,
                             BuildRepo,
                             BuildRepo2,
                             MetadataWrite,
                             RepoNameFixer,
                             repo_md,
                             GitenbergJobRunner,
                             MetadataWriterRunner,
                             RepoJobRunner,
                             StatusUpdateRunner)

from gitenberg_db import Repo, create_session

# metadatawrite

In [ ]:
mwr = MetadataWriterRunner("gitenberg.db", username, password)

In [ ]:
mwr.run(1)

In [ ]:
mwr.exceptions()

# Building books

In [ ]:
session = create_session("gitenberg.db")

In [ ]:
(session.query(Repo)
 .filter(or_(Repo.buildable == None, Repo.buildable == True))
 .filter(Repo.datebuilt == None)
 .filter(Repo.metadata_written != None)
).count()

In [ ]:
rjr = RepoJobRunner("gitenberg.db", username, password, GITENBERG_TRAVIS_ACCESS_TOKEN)

In [ ]:
rjr.run(10)

In [ ]:
rjr.exceptions()

In [ ]:
rjr.gh.ratelimit_remaining

In [ ]:
dt = arrow.get(rjr.gh.rate_limit()['rate']['reset']) - arrow.now()
rjr.countdown(dt.seconds)

# StatusUpdater

In [ ]:
(session.query(Repo)
 .filter(Repo.datebuilt != None)
 .filter(Repo.last_build_id == None)
).count()

In [ ]:
sur = StatusUpdateRunner("gitenberg.db", username, password, GITENBERG_TRAVIS_ACCESS_TOKEN)

In [ ]:
sur.exceptions()

# overall stats

In [ ]:
(session.query(Repo)
  .filter(Repo.ebooks_in_release_count == 3)
).count()

In [ ]:
session.query(Repo.ebooks_in_release_count).distinct().all()


```sql
SELECT ebooks_in_release_count, count (ebooks_in_release_count)
  FROM Repos
 GROUP BY ebooks_in_release_count
```

In [ ]:
# http://stackoverflow.com/a/4086229/7782

from sqlalchemy import func
(session.query(Repo.ebooks_in_release_count, func.count(Repo.ebooks_in_release_count))
   .group_by(Repo.ebooks_in_release_count).all())

In [ ]:
session.query(Repo).distinct(Repo.ebooks_in_release_count).count()

In [ ]:
sur.gh.ratelimit_remaining

In [ ]:
dt = arrow.get(sur.gh.rate_limit()['rate']['reset']) - arrow.now()
sur.countdown(dt.seconds)

# Misc

In [ ]:
for repo in session.query(Repo).filter_by(ebooks_in_release_count = 3):
    repo.has_metadata = True
    repo.has_source = True
    repo.buildable = True
    repo.updated = arrow.now().isoformat()


session.commit()

In [ ]:
import gitenberg

b = gitenberg.Book(1)
b.parse_book_metadata()
b.meta.metadata 


In [ ]:
import yaml
md = repo_md(1)
print (yaml.safe_dump(md,default_flow_style=False,
                                  allow_unicode=True))

In [ ]:
1/0

In [ ]:
def status_for_repo(repo_name):
    rs = GitenbergTravisJob(username=username, password=password, repo_name=repo_name,
              repo_owner='GITenberg', 
              update_travis_commit_msg='check status',
              tag_commit_message='check status',
              access_token=GITENBERG_TRAVIS_ACCESS_TOKEN)
    return rs.status()

results_iter = apply_to_repos(status_for_repo, repos=all_repos)

In [ ]:
results = []

for (i,result) in enumerate(results_iter):
    
    results.append(result)
    if not isinstance(result, Exception):
        print ("\r{}: {}".format(i, result['repo_name']), end="")
    else:
        print ("\r{}: {}".format(i, str(result)), end="")

In [ ]:
[(i, result) for (i, result) in enumerate(results) if isinstance(result, Exception)]

In [ ]:
[result.get('repo_name') for result in results if result.get('ebooks_in_release_count') != 3]

In [ ]:
# update the database based on result

result = results[0]
result

In [ ]:
for result in results:
    repo = session.query(Repo).filter_by(repo_name=result['repo_name']).first()
    repo.updated = arrow.now().isoformat()
    repo.datebuilt = result['last_build_started_at']
    repo.version = result['version']
    repo.ebooks_in_release_count = result['ebooks_in_release_count']
    repo.last_build_id = result['last_build_id']
    repo.last_build_state = result['last_build_state']
    
session.commit()
    

In [ ]:
# building the rest

session.query(Repo).filter(Repo.datebuilt != None).count()

In [ ]:
repo_names = [repo.repo_name for repo in 
islice(session.query(Repo).filter(Repo.datebuilt == None).order_by(Repo.gutenberg_id.asc()),5)]

In [ ]:
from collections import OrderedDict
from itertools import islice

results = OrderedDict()

In [ ]:
repos_iter = iter(repo_names)

In [ ]:
def build_repos(repo_names, n=None):
    for (i, repo_name) in enumerate(islice(repo_names, n)):
        try:
            bj = BuildRepo2(username=username, password=password, repo_name=repo_name,
                  repo_owner='GITenberg', 
                  update_travis_commit_msg='build using gitenberg.travis',
                  tag_commit_message='build using gitenberg.travis',
                  access_token=GITENBERG_TRAVIS_ACCESS_TOKEN)
            results[repo_name] = (bj, bj.run())
            
            # just mark as started
            repo = session.query(Repo).filter_by(repo_name=result['repo_name']).first()
            repo.updated = arrow.now().isoformat()
            repo.datebuilt = arrow.now().isoformat()
            
        except Exception, e:
            results[repo_name] = e
        
        print ("\r{}: {}".format(i, results[repo_name]), end="")

In [ ]:
build_repos(repos_iter, 1)

wondering if not add_all -- any add or update function?  [python - SQLAlchemy insert or update example - Stack Overflow](http://stackoverflow.com/questions/7889183/sqlalchemy-insert-or-update-example)

In [ ]:
repo1.version = '0.0.5'

In [ ]:
session.dirty

In [ ]:
session.new

In [ ]:
our_repo = session.query(Repo).filter_by(repo_name='Repo1').first() # doctest:+NORMALIZE_WHITESPACE
our_repo
